In [ ]:
import re
import json
import pandas as pd
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import Document
from langchain.memory import ConversationBufferMemory, ChatMessageHistory
from langchain.chains import ConversationalRetrievalChain
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#set API Key from OpenAI
openai_api_key= "Add Your OpenAI API KEY Here."

In [ ]:
def LCI_pathway_extraction(table_title, page_number, persist_directory):
    """Extract the metadata (i.e., pathway and activity) from the inventory table."""

    #create a Chroma vector store, specifying the persistence directory
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"filter": {"page_number": page_number}})

    #initial the llm
    llm = ChatOpenAI(
        model_name="gpt-4o",
        temperature=0,
        max_tokens=4000,
        openai_api_key=openai_api_key
    )

    #initial the memory
    msgs = ChatMessageHistory()
    memory = ConversationBufferMemory(memory_key="chat_history", chat_memory=msgs, return_messages=True)

    #initial the prompt
    template1 = """
        You are a helpful assistant specializing in extracting life cycle assessment (LCA) data. Based on the provided context, use the definitions below to answer the question at the end. If you are unsure of the answer, state "I don't know" rather than conjecture.

        ### Extract the information based on the definitions below: 
        - **Pathway:** The sequence of processes and transformations that raw materials undergo to become a final product. This includes all intermediate steps, inputs, and outputs. Synonyms include "Processes," "Technical route," "Scenario," "Conversion." 
        - **Activity:** A unit process of converting raw materials into intermediates or products. It represents the smallest element in the product system for which input and output data are quantified. Examples include raw material production, transport, assembly, synthesis, production, and end-of-life treatment. 

        ### Instructions for Data Extraction: 
        - Identify and list the pathways and activities with minimum detail to understand their relationships. 
        - Only extract the pathway and activity from the given table's meta. Ensure completeness and precision in your extraction. 
        - Avoid extra explanations and format your response as JSON format follows: {{"Pathway": ["Activity"], "Pathway": ["Activity"]}}
            - If you do not find any information about the pathway or activity, label them as "Not mentioned."

        ###Examples:
        1. Table: <The inventory data for PV/CCU-CH 3 OH technical route\n\nInputs Values Units Outputs Values Units\n\n**Hydrogen generation**\nElectricity 10070 kWh Hydrogen 190 kg\n\n
        Water 1710 MJ Waste water 87.4 kg\n\nPotassium hydroxide 0.57 kg Oxygen 1501 kg\n\nHydrogen plant 1.02E-6 Unit\n\n**Methanol synthesis and purification**\nElectricity 42.75 kWh Methanol 1000 kg\n\nHydrogen 190 MJ Waste water 630 kg\n\n
        Carbon dioxide (HP) 1455 kg\n\nAluminium oxide 0.012 kg\n\nCopper oxide 0.062 kg\n\nZinc oxide 0.029 kg>
        Answer: 
        - “Pathway”: [“PV/CCU-CH 3 OH technical route”]
        - “Activity”: [“Hydrogen generation”, “Methanol synthesis and purification”]
        2. Table: <Table. Life Cycle Energy Consumption Inventory Data of the CtEG Route for Producing 1 ton EG\n\nparameter unit coal mining and processing coal transportation coal to EG total life cycle\n\nMaterials Consumption [8]\n\ncoal t 3.17 × 10 [0] 3.17 × 10 [0]\n\n
        Energy Consumption [8] [,] [22] [,] [31]\n\nfuel coal GJ 2.89 × 10 [0] 4.73 × 10 [1] 5.02 × 10 [1]\n\ndiesel GJ 5.79 × 10 [−] [2] 2.47 × 10 [−] [1] 3.05 × 10 [−] [1]\n\ngasoline GJ 5.79 × 10 [−] [2] 1.73 × 10 [−] [2] 7.52 × 10 [−] [2]\n\nelectricity GJ 5.79 × 10 [−] [1] 1.47 × 10 [−] [1] 3.16 × 10 [0] 3.89 × 10 [0]\n\n
        Pollutant Emission [8] [,] [22] [,] [32]\n\nCO 2 kg 1.26 × 10 [2] 6.30 × 10 [1] 6.72 × 10 [3] 6.91 × 10 [3]\n\nCH 4 kg 2.37 × 10 [0] 3.42 × 10 [−] [1] 2.44 × 10 [1] 2.71 × 10 [1]\n\nN 2 O kg 2.13 × 10 [−] [3] 1.47 × 10 [−] [4] 7.69 × 10 [−] [3] 9.97 × 10 [−] [3]\n\nCO kg 1.51 × 10 [−] [2] 5.41 × 10 [−] [1] 1.06 × 10 [1] 1.12 × 10 [1]\n\n
        NO x kg 4.67 × 10 [−] [1] 1.17 × 10 [−] [1] 3.53 × 10 [0] 4.11 × 10 [0]\n\nSO 2 kg 1.69 × 10 [−] [1] 2.69 × 10 [−] [2] 1.44 × 10 [0] 1.64 × 10 [0]\n\nPM 10 kg 5.23 × 10 [−] [1] 1.08 × 10 [−] [1] 2.92 × 10 [0] 3.55 × 10 [0]\n\nVOC kg 8.24 × 10 [−] [3] 8.00 × 10 [−] [3] 8.58 × 10 [−] [2] 1.02 × 10 [−] [1]\n\n>
        Answer: 
        - “Pathway”: [“CtEG Route”]
        - “Activity”: [“Coal mining and processing”, “coal mining and processing”, “ Coal to EG”]
        3. Tanle: <Table.** Inventory to produce 1 metric ton of EG via different pathways[14, 18]\n\nUnit Petro-EG Coal-EG\n\nFeedstock\n\nCoal t 3.17\n\nMethanol t 0.60\n\nNH 3 t 0.30\n\nEthylene t 0.75\n\nWater t 3\n\nUtilites\n\nElectricity kWh 300 1200\n\n4.2 MPa steam t 0.10 0.42\n\n
        1.5 MPa steam t 0.40 4.50\n\n0.5 MPa steam t 0.10 3.90\n\nDirect CO 2 emissions t 0.94 5.44\n\n17\n>
        Answer:
        - "Pathway": ["Petro-EG", "Coal-EG"]
        - "Activity": [ ]

        {chat_history}
        {context}
        Question: {question}
        """

    prompt1 = PromptTemplate(
        template=template1, input_variables=["context", "chat_history", "question"], output_parser=CommaSeparatedListOutputParser()
    )

    #set up chain
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=False,
        memory=memory,
        combine_docs_chain_kwargs={'prompt': prompt1}
    )

    #get the result
    result1 = []

    query1 = f"extract the Pathway and Activity from the [{table_title}]."
    result = chain({"question": query1})
    # Parse the model's string output into a Python dictionary
    extracted = result["answer"].strip('```json').strip('```').strip()
    try:
        result_dict = json.loads(extracted)
    except json.JSONDecodeError:
        result_dict = {"Pathway": ["Not mentioned"], "Activity": ["Not mentioned"]}

    return result_dict, retriever

def LCI_data_extraction(table_title, page_number, pathway, activity, persist_directory):
    """Extract LCI data from the inventory table."""

    #create a Chroma vector store, specifying the persistence directory
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"filter": {"page_number": page_number}})
    
    #initial the llm
    llm = ChatOpenAI(
        model_name="gpt-4o",
        temperature=0,
        max_tokens=4000,
        openai_api_key=openai_api_key
    )

    #initial the memory
    msgs = ChatMessageHistory()
    memory = ConversationBufferMemory(memory_key="chat_history", chat_memory=msgs, return_messages=True)

    #initial the prompt
    template2 = """
        You are a helpful assistant specializing in extracting life cycle assessment (LCA) data. Based on the provided context, use the definitions below to answer the question at the end. If you are unsure of the answer, state "I don't know" rather than conjecture.

        ### Extract the information based on the definitions below: 
        - **Inputs**: is defined as input substances used to manufacture intermediates or products. Input materials generally include human-made systems and resources from the environment. The examples of inputs are feedstocks, energy, natural resources, intermediates, devices, transports, solvents, and chemicals. These input materials are quantified by a numeric value with a unit.
        - **Outputs**: is defined as output substances from the production activity. Output materials typically include emissions, wastes, and output products. These outputs are also quantified using specific numeric values with units.

        ### Instructions for data Extraction: 
        - Identify and list the inputs and outputs with minimum detail to understand their relationships. 
        - Inputs include both human-made products like raw materials, chemicals, intermediates, products, components, devices, solvents, electricity,  fuel and resources from the environment like water.
        - Outputs mainly include emissions to air, emissions to water, wastes to treatment, and output products.
        - Extract all parameters related to each input and output category from the given context. Ensure completeness and precision in your extraction. 
        - If the input or output has multiple distinct values, each value should be represented by a separate input/output entity.
        - The value of the input and output must be numeric and accompanied by a unit, and cannot be a string or text
        - Avoid extra explanations and format your response as follows:
            - "Inputs": [] # list format 
            - "Outputs": [] # list format 
        - If you do not find any information about the input and outp, label them as "Not mentioned."

        ### Steps of data Extraction: 
        1. Understand the format of the markdown table, including the meaning of each column and row.
        2. Based on the provided pathway and activity, extract all relevant inputs and outputs within the table.

        ###Examples:
        1. Question - Extract inputs and outputs of activity “Hydrogen generation” of pathway “PV/CCU-CH 3 OH technical route” from the Table: <The inventory data for PV/CCU-CH 3 OH technical route\n\nInputs Values Units Outputs Values Units\n\n**Hydrogen generation**\n
        Electricity 10070 kWh Hydrogen 190 kg\n\nWater 1710 MJ Waste water 87.4 kg\n\nPotassium hydroxide 0.57 kg Oxygen 1501 kg\n\nHydrogen plant 1.02E-6 Unit\n\n**Methanol synthesis and purification**\nElectricity 42.75 kWh Methanol 1000 kg\n\nHydrogen 190 MJ Waste water 630 kg\n\nCarbon dioxide (HP) 1455 kg\n\n
        Aluminium oxide 0.012 kg\n\nCopper oxide 0.062 kg\n\nZinc oxide 0.029 kg>
        Answer: 
        - “Inputs”: [“Electricity”: 10070 kWh, “Water”: 1710 MJ, “Potassium”: 0.57 kg, “Hydrogen plant”: 1.02E-6 Unit]
        - “Outputs”: [“Hydrogen”: 190 kg, “Waste water”: 87.4 kg, “Oxygen”: 1501 kg]
        2. Question - Extract inputs and outputs of activity “coal to EG” of pathway “CtEG route” from the Table: <Table. Life Cycle Energy Consumption Inventory Data of the CtEG Route for Producing 1 ton EG\n\nparameter unit coal mining and processing coal transportation coal to EG total life cycle\n\nMaterials Consumption [8]\n\n
        coal t 3.17 × 10 [0] 3.17 × 10 [0]\n\nEnergy Consumption [8] [,] [22] [,] [31]\n\nfuel coal GJ 2.89 × 10 [0] 4.73 × 10 [1] 5.02 × 10 [1]\n\ndiesel GJ 5.79 × 10 [−] [2] 2.47 × 10 [−] [1] 3.05 × 10 [−] [1]\n\ngasoline GJ 5.79 × 10 [−] [2] 1.73 × 10 [−] [2] 7.52 × 10 [−] [2]\n\nelectricity GJ 5.79 × 10 [−] [1] 1.47 × 10 [−] [1] 3.16 × 10 [0] 3.89 × 10 [0]\n\n
        Pollutant Emission [8] [,] [22] [,] [32]\n\nCO 2 kg 1.26 × 10 [2] 6.30 × 10 [1] 6.72 × 10 [3] 6.91 × 10 [3]\n\nCH 4 kg 2.37 × 10 [0] 3.42 × 10 [−] [1] 2.44 × 10 [1] 2.71 × 10 [1]\n\nN 2 O kg 2.13 × 10 [−] [3] 1.47 × 10 [−] [4] 7.69 × 10 [−] [3] 9.97 × 10 [−] [3]\n\nCO kg 1.51 × 10 [−] [2] 5.41 × 10 [−] [1] 1.06 × 10 [1] 1.12 × 10 [1]\n\nNO x kg 4.67 × 10 [−] [1] 1.17 × 10 [−] [1] 3.53 × 10 [0] 4.11 × 10 [0]\n\n
        SO 2 kg 1.69 × 10 [−] [1] 2.69 × 10 [−] [2] 1.44 × 10 [0] 1.64 × 10 [0]\n\nPM 10 kg 5.23 × 10 [−] [1] 1.08 × 10 [−] [1] 2.92 × 10 [0] 3.55 × 10 [0]\n\nVOC kg 8.24 × 10 [−] [3] 8.00 × 10 [−] [3] 8.58 × 10 [−] [2] 1.02 × 10 [−] [1]\n\n>
        - “Inputs”: [“coal”: 3.17 t, “fuel coal”: 47.3 GJ, “electricity”: 3.16 GJ]
        - “Outputs”: [“CO2”: 6720 kg, “CH4”: 24.4 kg, “N2O”: 0.00769 kg, “CO”: 10.6 kg, “NOx”: 3.53 kg, “SO2”: 1.44 kg, “PM10”: 2.92 kg, “VOC”: 0.0858 kg]
        3. Question - Extract inputs and outputs of activity “” of pathway “Petro-EG” from the Table: <Table.** Inventory to produce 1 metric ton of EG via different pathways[14, 18]\n\nUnit Petro-EG Coal-EG\n\nFeedstock\n\nCoal t 3.17\n\nMethanol t 0.60\n\nNH 3 t 0.30\n\nEthylene t 0.75\n\nWater t 3\n\nUtilites\n\nElectricity kWh 300 1200\n\n4.2 MPa steam t 0.10 0.42\n\n
        1.5 MPa steam t 0.40 4.50\n\n0.5 MPa steam t 0.10 3.90\n\nDirect CO 2 emissions t 0.94 5.44\n\n17\n>
        Answer:
        - “Inputs”: [“Ethylene”: 0.75 t, “Water”: 3 t, “Electricity”: 300 kWh, “4.2 MPa steam”: 0.1 t, “1.5 MPa steam”: 0.4 t, “0.5 MPa steam”: 0.1 t]
        - “Outputs”: [“Direct CO2 emissions”: 0.94 t]


        {chat_history}
        {context}
        Question: {question}
        """

    prompt2 = PromptTemplate(
        template=template2, input_variables=["context", "chat_history", "question"], output_parser=CommaSeparatedListOutputParser()
    )

    #set up chain
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=False,
        memory=memory,
        combine_docs_chain_kwargs={'prompt': prompt2}
    )

    #get the result
    result2 = []

    query2 = f"Extract inputs and outputs of the activity {activity} from the pathway {pathway} from the [{table_title}]."
    result = chain({"question": query2})
    result2.append(result["answer"].strip('```json').strip('```').strip())
    
    msgs.clear()
    memory.clear()
    
    return result2

def convert_to_dataframe (raw_text):
    """Convert raw_LCI data into the structured dataframe."""
    
    llm = ChatOpenAI(
        model_name="gpt-4o",
        temperature=0,
        max_tokens=4000,
        openai_api_key=openai_api_key
    )
    
    template3 = """
    You are a helpful assistant specializing in converting life cycle assessment (LCA) data format. Based on the provided context, use the definitions below to answer the question at the end.

    ### Convert the provided list into a structured format based on the definitions below: 
    - **Flow name**: the name of the flow.
    - **Input/Outputs**: specifies whether it is an input or an output flow.
    - **Amount**: the quantity of the flow.
    - **Unit**: the unit in which the amount is measured.
    
    ###Avoid extra explanations and format your response as JSON format following:[{{"Flow name": "", "Input/Output": "", "Amount": "", "Unit": ""}}, {{"Flow name": "", "Input/Output": "", "Amount": "", "Unit": ""}}]

    ###Examples:
    1. Question: <['- "Inputs": ["Electricity": 10070 kWh, "Water": 1710 MJ, "Potassium hydroxide": 0.57 kg, "Hydrogen plant": 1.02E-6 Unit]\n- "Outputs": ["Hydrogen": 190 kg, "Waste water": 87.4 kg, "Oxygen": 1501 kg]']>
    Answer: 
    [
    {{"Flow name": "Electricity", "Input/Output": "Input", "Amount": 10070, "Unit": "kWh"}},
    {{"Flow name": "Water", "Input/Output": "Input", "Amount": 1710, "Unit": "MJ"}},
    {{"Flow name": "Potassium hydroxide", "Input/Output": "Input", "Amount": 0.57, "Unit": "kg"}},
    {{"Flow name": "Hydrogen plant", "Input/Output": "Input", "Amount": 1.02e-6, "Unit": "Unit"}},
    {{"Flow name": "Hydrogen", "Input/Output": "Output", "Amount": 190, "Unit": "kg"}},
    {{"Flow name": "Waste water", "Input/Output": "Output", "Amount": 87.4, "Unit": "kg"}},
    {{"Flow name": "Oxygen", "Input/Output": "Output", "Amount": 1501, "Unit": "kg"}}
    ]
    2. Question: <['- "Inputs": ["coal": 3.17 t, "fuel coal": 50.2 GJ, "diesel": 0.0305 GJ, "gasoline": 0.00752 GJ, "electricity": 3.16 GJ]\n- "Outputs": ["CO2": 6720 kg, "CH4": 24.4 kg, "N2O": 0.00769 kg, "CO": 10.6 kg, "NOx": 3.53 kg, "SO2": 1.44 kg, "PM10": 2.92 kg, "VOC": 0.0858 kg]']>
    Answer: 
    [
    {{"Flow name": "Coal", "Input/Output": "Input", "Amount": 3.17, "Unit": "t"}},
    {{"Flow name": "Fuel Coal", "Input/Output": "Input", "Amount": 50.2, "Unit": "GJ"}},
    {{"Flow name": "Diesel", "Input/Output": "Input", "Amount": 0.0305, "Unit": "GJ"}},
    {{"Flow name": "Gasoline", "Input/Output": "Input", "Amount": 0.00752, "Unit": "GJ"}},
    {{"Flow name": "Electricity", "Input/Output": "Input", "Amount": 3.16, "Unit": "GJ"}},
    {{"Flow name": "CO2", "Input/Output": "Output", "Amount": 6720, "Unit": "kg"}},
    {{"Flow name": "CH4", "Input/Output": "Output", "Amount": 24.4, "Unit": "kg"}},
    {{"Flow name": "N2O", "Input/Output": "Output", "Amount": 0.00769, "Unit": "kg"}},
    {{"Flow name": "CO", "Input/Output": "Output", "Amount": 10.6, "Unit": "kg"}},
    {{"Flow name": "NOx", "Input/Output": "Output", "Amount": 3.53, "Unit": "kg"}},
    {{"Flow name": "SO2", "Input/Output": "Output", "Amount": 1.44, "Unit": "kg"}},
    {{"Flow name": "PM10", "Input/Output": "Output", "Amount": 2.92, "Unit": "kg"}},
    {{"Flow name": "VOC", "Input/Output": "Output", "Amount": 0.0858, "Unit": "kg"}}
    ]
    3. Question: <['- "Inputs": ["Coal": 3.17 t, "Methanol": 0.60 t, "NH3": 0.30 t, "Ethylene": 0.75 t, "Water": 3 t, "Electricity": 300 kWh, "4.2 MPa steam": 0.10 t, "1.5 MPa steam": 0.40 t, "0.5 MPa steam": 0.10 t]\n- "Outputs": ["Direct CO2 emissions": 0.94 t]']>
    Answer: 
    [
    {{"Flow name": "Coal", "Input/Output": "Input", "Amount": 3.17, "Unit": "t"}},
    {{"Flow name": "Methanol", "Input/Output": "Input", "Amount": 0.60, "Unit": "t"}},
    {{"Flow name": "NH3", "Input/Output": "Input", "Amount": 0.30, "Unit": "t"}},
    {{"Flow name": "Ethylene", "Input/Output": "Input", "Amount": 0.75, "Unit": "t"}},
    {{"Flow name": "Water", "Input/Output": "Input", "Amount": 3, "Unit": "t"}},
    {{"Flow name": "Electricity", "Input/Output": "Input", "Amount": 300, "Unit": "kWh"}},
    {{"Flow name": "4.2 MPa steam", "Input/Output": "Input", "Amount": 0.10, "Unit": "t"}},
    {{"Flow name": "1.5 MPa steam", "Input/Output": "Input", "Amount": 0.40, "Unit": "t"}},
    {{"Flow name": "0.5 MPa steam", "Input/Output": "Input", "Amount": 0.10, "Unit": "t"}},
    {{"Flow name": "Direct CO2 emissions", "Input/Output": "Output", "Amount": 0.94, "Unit": "t"}}
    ]
    """
        
    prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", template3),
        ("human", "Respond to question: {question}")
    ])

    # Insert a question into the template and call the model
    result3 = []
    
    query3 = f"Convert the list {raw_text} into defined data format"
    full_prompt = prompt_template.format_messages(question=query3)
    result = llm.invoke(full_prompt)
    result3.append(result)

    data = result3[0].content
    
    return data

def data_to_dataframe(data):
    """Convert a JSON-formatted string into a pandas DataFrame."""
    
    data = data.strip()
    if data.startswith('```json'):
        data = data[7:]  
    if data.startswith('```'):
        data = data[3:]  
    if data.endswith('```'):
        data = data[:-3]
    data = data.strip()
    
    data_list = json.loads(data)

    df = pd.DataFrame(data_list)
    return df

def process_document (df_table_title, df_vectordb):
    """Merge and group table metadata."""
    
    df_main = pd.merge(df_table_title, df_vectordb, on='PDF_path', how='inner')
    df_main = df_main.drop(["Num_pages", "Num_chunks", "Is_added_to_vectorDB"], axis=1)

    df_grouped = {table: df_group for table, df_group in df_main.groupby("Paper_title", sort=False)}

    return df_grouped

def process_table(df_paper):
    """Extract pathways and activities for each inventory table in a paper."""
    
    table_info = []

    for _, row in df_paper.iterrows():
        table_title = row['Table_title']
        persist_directory = row['Vectordb_path']
        page_number = row['Page']

        data_dict, retriever = LCI_pathway_extraction(table_title, page_number, persist_directory)

        pathways = data_dict.get("Pathway", [])
        activities = data_dict.get("Activity", [])

        # Ensure both are lists
        if isinstance(pathways, str):
            pathways = [pathways]
        if isinstance(activities, str):
            activities = [activities]

        if not pathways or not activities:
            continue  # Skip if missing

        pathway = pathways[0]

        rows = [(pathway, activity) for activity in activities]

        df_table = pd.DataFrame(rows, columns=["Pathway", "Activity"])
        df_table["Paper_title"] = row["Paper_title"]
        df_table["Table_title"] = table_title
        df_table["Vectordb_path"] = persist_directory
        df_table["Page"] = page_number

        table_info.append(df_table)

    df_table = pd.concat(table_info, ignore_index=True)

    return df_table, retriever

def extract_LCI(df_table, retriever):
    """Extract LCI data.""" 
    LCI_db = {}  # Dictionary to store extracted LCI data

    for _, row in df_table.iterrows():
        table_title = row["Table_title"]
        pathway = row["Pathway"]
        activity = row["Activity"]
        persist_directory = row['Vectordb_path']
        page_number = row['Page']

        print(f"Processing Table: {table_title}, Pathway: {pathway}, Activity: {activity}")

        # Extract raw LCI data
        raw_data = LCI_data_extraction(table_title, page_number, pathway, activity, persist_directory)

        # Convert raw data into a DataFrame
        data = convert_to_dataframe(raw_data)
        df_LCI = data_to_dataframe(data)

        # Add metadata to LCI data
        df_LCI["Pathway"] = pathway
        df_LCI["Activity"] = activity

        # Append data to the existing table title if already present
        if table_title in LCI_db:
            LCI_db[table_title] = pd.concat([LCI_db[table_title], df_LCI], ignore_index=True)
        else:
            LCI_db[table_title] = df_LCI

    return LCI_db

def save_LCI_db_to_csv(LCI_db, output_file):
    """Save extracted LCI data to a CSV file."""
    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        for table_title, df in LCI_db.items():
            f.write(f"# {table_title}\n")  
            df = df.dropna(how='all') 
            df.to_csv(f, index=False)  

In [ ]:
# Input path to the vector DB index CSV generated in Step 1
df_vectordb_path = "Add your output CSV path here (from Step 1)"
# Input path to the table title classification CSV generated in Step 3.1
df_table_title_path = "Add your output CSV path here (from Step 3.1)"

df_vectordb = pd.read_csv(df_vectordb_path, encoding='ISO-8859-1')
df_table_title = pd.read_csv(df_table_title_path, encoding='ISO-8859-1')

# Group table data by paper
df_grouped = process_document(df_table_title, df_vectordb)
paper_title = list(df_grouped.keys())
df_paper_title = pd.DataFrame(paper_title, columns=['Paper_title'])
df_paper_title

In [ ]:
# Select a single paper (row index can be adjusted as needed)
paper_title = list(df_grouped.keys())[0]
df_paper = df_grouped[paper_title]
df_paper

In [ ]:
# Extract the metadata (i.e., pathway and activity) from inventory tables
df_table, retriever = process_table(df_paper)
df_table

In [ ]:
# Extract LCI data
LCI_db = extract_LCI (df_table, retriever)

In [ ]:
# Search LCI data for a table (row index can be adjusted as needed)
key_list = list(LCI_db.keys())
key = key_list[0]
df_LCI = LCI_db.get(key, "No data found")
df_LCI

**2. Human-in-loop option**

In [ ]:
# Load required fields from df_table (row index can be adjusted as needed)
table_title = df_table["Table_title"][0]
page_number = int(df_table["Page"][0])
vectordb_path = df_table["Vectordb_path"][0]

In [ ]:
# Table metadata (loaded from df_table)
table_title = table_title
page_number = page_number
vectordb_path = vectordb_path
# User-defined pathway (see definition in Prompt Template 1)
pathway = "Add pathway here"
# User-defined activity (see definition in Prompt Template 1)
activity = "Add activity here"
 
raw_data = LCI_data_extraction(table_title, page_number, pathway, activity, vectordb_path)
data = convert_to_dataframe(raw_data)
df_LCI = data_to_dataframe(data)
df_LCI

**3. Save the extracted data**

In [ ]:
# Output file path for saving the extracted LCI data
file_name = "Add your output CSV path here" 
save_LCI_db_to_csv(LCI_db, file_path)